In [133]:
import numpy as np
import random
from collections import deque
import gym

In [134]:
import tensorflow as tf

In [135]:
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam


In [136]:
env_name = 'CartPole-v1'

In [137]:
env = gym.make(env_name)

In [138]:
# env.reset()
# for step in range(500):
#     env.render(mode='human')
#     random_action = env.action_space.sample()
#     env.step(random_action)
    
# env.close()

In [139]:
# 4 observations we have in cartpole
env.observation_space.shape

(4,)

In [140]:
num_observations = env.observation_space.shape[0]

In [141]:
env.observation_space.shape

(4,)

In [142]:
num_observations

4

In [143]:
num_actions = env.action_space.n

In [144]:
num_actions

2

In [145]:
# input shape of the ANN should be equal to number of observation --> 4 here
# output size of the ANN should be equal to number of actions --> 2 here

In [148]:
model = Sequential()
# by adding the input_shape we become sure that we match currently to the firt layer regardless of num of obs
model.add(Dense(16, input_shape=(1, num_observations)))
# model.add(Dense(16))

model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))


# Neurons == action_space
model.add(Dense(num_actions))
model.add(Activation('linear'))

In [149]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 1, 16)             80        
                                                                 
 activation_12 (Activation)  (None, 1, 16)             0         
                                                                 
 dense_14 (Dense)            (None, 1, 32)             544       
                                                                 
 activation_13 (Activation)  (None, 1, 32)             0         
                                                                 
 dense_15 (Dense)            (None, 1, 2)              66        
                                                                 
 activation_14 (Activation)  (None, 1, 2)              0         
                                                                 
Total params: 690
Trainable params: 690
Non-trainable 

In [150]:
target_model = clone_model(model)

In [151]:
# Now these tasks ought to be done:
#     1- Define Hyper parameters
#     2- Epsilon greedy action selection
#     3- Undersatnd Deque Object
#     4- Create Replay Function
#     5- Tartget Model Update Function

In [152]:
EPOCHS = 1000
BATCH_SIZE = 32

epsilon = 1.0
EPSILON_REDUCE = 0.995

LEARNIN_RATE = 0.001
GAMMA = 0.95

In [223]:
def epsilon_greedy_action_selection(model, epsilon, observation, batch_size=32):
    if np.random.random() > epsilon:
        print("zzzz")
        print("observation: ", observation)
        print("observation Type: ", type(observation))
        print("observation shape: ", observation.shape)
        print("list obs: ", list(observation))
        prediction = model.predict(observation.reshape([1, 1, 4]))
        print("wwww")
        action = np.argmax(prediction)
        print("hhhh")
    else:
        print("ssss")
        action = np.random.randint(0, env.action_space.n)
        
    return action

In [202]:
# deque add a limit to the number of values that can be added to it and once it reach the threshold it will stop adding elements
# and if continue appending items to it it will remove the very first one and add th last one
deque_1 = deque(maxlen=5)

In [203]:
deque_1

deque([])

In [204]:
for i in range(10):
    deque_1.append(i)
    print(deque_1)

deque([0], maxlen=5)
deque([0, 1], maxlen=5)
deque([0, 1, 2], maxlen=5)
deque([0, 1, 2, 3], maxlen=5)
deque([0, 1, 2, 3, 4], maxlen=5)
deque([1, 2, 3, 4, 5], maxlen=5)
deque([2, 3, 4, 5, 6], maxlen=5)
deque([3, 4, 5, 6, 7], maxlen=5)
deque([4, 5, 6, 7, 8], maxlen=5)
deque([5, 6, 7, 8, 9], maxlen=5)


In [205]:
# We use the deque to define replay buffer
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [206]:
def replay(replay_buffer, batch_size, model, target_model):
    if len(replay_buffer)<batch_size:
        return
   
    samples = random.sample(replay_buffer, batch_size)

    target_batch = list()
    zipped_samples = list(zip(*samples))

    states, actions, rewards, new_states, dones = zipped_samples
    print(np.shape(np.array(states)))
    print(4)
    states = np.array(states).reshape([batch_size, 1, 4])
    new_states = np.array(new_states).reshape([batch_size, 1, 4])
    targets = target_model.predict(states)
    # predict q values for all the samples
    print(5)
#     q_values = model.predict(np.array(new_states))
    q_values = model.predict(new_states)

    print(6)
    for i in range(batch_size):
        print(7)
        q_value = max(q_values[i][0])
        print(8)
        target = targets[i].copy()
        print(9)
        if dones[i]:
            print(10)
            target[0][actions[i]] = rewards[i]
        else:
            print(12)
            target[0][actions[i]] = rewards[i]+q_value*GAMMA
        print(13)
        target_batch.append(target)
    
    print(14)
    print("np.array(states).shap: ", np.array(states).shape)
    print("np.array(target_batch).shape: ", np.array(target_batch).shape)
    model.fit(np.array(states), np.array(target_batch), epochs=1, verbose=0)
    

In [207]:
def update_model_handler(epoch, update_target_model, model, target_moel):
    if epoch > 0 and epoch%update_target_model==0:
        target_model.set_weights(model.get_weights())

In [208]:
model.compile(loss='mse', optimizer=(Adam(lr=LEARNIN_RATE)))

In [209]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 1, 16)             80        
                                                                 
 activation_12 (Activation)  (None, 1, 16)             0         
                                                                 
 dense_14 (Dense)            (None, 1, 32)             544       
                                                                 
 activation_13 (Activation)  (None, 1, 32)             0         
                                                                 
 dense_15 (Dense)            (None, 1, 2)              66        
                                                                 
 activation_14 (Activation)  (None, 1, 2)              0         
                                                                 
Total params: 690
Trainable params: 690
Non-trainable 

In [224]:
best_so_far = 0

for epoch in range(EPOCHS):
    print("00")
    observation = env.reset()
    # (1, X) X observation size here is 4 we resgape to be (1, 4)
    obsrvation = observation.reshape([1, 4])
    done = False
    
    points = 0
    
    while not done:
        print("000")
        print("observation: ", observation)
        print("model: ", model)
        action = epsilon_greedy_action_selection(model, epsilon, observation)
        print("111")
        next_observation, reward, done, info = env.step(action)
        print("222")
        next_observation = next_observation.reshape([1, 4])
        print("333")
        replay_buffer.append((observation, action, reward, next_observation, done))
        print("444")
        observatin = next_observation
        
        points += 1
        
        print("I am Here .... ")
        replay(replay_buffer, BATCH_SIZE, model, target_model)
        print("5555")
    
    print("I am NOW INJA .... ")
    epsilon *= EPSILON_REDUCE # eps*0.995
    print("6666")
    update_model_handler(epoch, update_target_model, model, target_model)
    print("7777")
    if points > best_so_far:
        best_so_far = points
        
    if epoch%25 == 0:
        print(f"{epoch}: POINTS: {points} eps: {epsilon} BSF: {best_so_far}")

00
000
observation:  [-0.04400268  0.00593079  0.03885338  0.03315154]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [-0.04400268  0.00593079  0.03885338  0.03315154]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
1

(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [-0.04400268  0.00593079  0.03885338  0.03315154]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13


6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [-0.02458906  0.00773078  0.04012631 -0.00716024]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13


6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [ 0.0339566  -0.03344926 -0.01203965  0.04916653]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13


6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [-0.03031913  0.01184305  0.00744087  0.00086668]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13


...

6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [ 0.03100954  0.02113285  0.01211943 -0.04544635]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13


9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [-4.22792248e-02  4.93955304e-05  1.42265055e-02  4.70321663e-02]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).s

6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [ 0.03494105  0.04588845 -0.00583241 -0.02914206]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
ssss
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
10
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13


9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
14
np.array(states).shap:  (32, 1, 4)
np.array(target_batch).shape:  (32, 1, 2)
5555
000
observation:  [ 0.03494105  0.04588845 -0.00583241 -0.02914206]
model:  <keras.engine.sequential.Sequential object at 0x000002889A2393D0>
zzzz
observation:  [ 0.03494105  0.04588845 -0.00583241 -0.02914206]
observation Type:  <class 'numpy.ndarray'>
observation shape:  (4,)
list obs:  [0.034941047, 0.04588845, -0.0058324113, -0.029142065]
wwww
hhhh
111
222
333
444
I am Here .... 
(32, 4)
4
5
6
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7
8
9
12
13
7


KeyboardInterrupt: 